In [1]:
# 1_Data_Cleaning.ipynb

# ---------------------------
# Step 1: Import Libraries
# ---------------------------
import pandas as pd
import numpy as np

# Define the base URL for raw data from the GitHub repository
# (Replace <USERNAME> with your actual GitHub username if needed)
url_base = "https://raw.githubusercontent.com/zingeroff/Restaurant_Sales_Analysis/main/data"

# ---------------------------
# Step 2: Load CSV Files
# ---------------------------

# Load guests data
guests_df = pd.read_csv(f"{url_base}/guests.csv")

# Load menu data
menu_df = pd.read_csv(f"{url_base}/menu-data.csv")

# Load monthly orders (April, May, June)
orders_apr_df = pd.read_csv(f"{url_base}/orders_04_2022.csv")
orders_may_df = pd.read_csv(f"{url_base}/orders_05_2022.csv")
orders_jun_df = pd.read_csv(f"{url_base}/orders_06_2022.csv")

print("Data files loaded successfully!")


Data files loaded successfully!


In [2]:
# ---------------------------
# Step 3: Initial Overview
# ---------------------------

print("Guests DataFrame Info:")
guests_df.info()
print("\n", guests_df.head(), "\n")

print("Menu DataFrame Info:")
menu_df.info()
print("\n", menu_df.head(), "\n")

print("Orders (April) Info:")
orders_apr_df.info()
print("\n", orders_apr_df.head(), "\n")

print("Orders (May) Info:")
orders_may_df.info()
print("\n", orders_may_df.head(), "\n")

print("Orders (June) Info:")
orders_jun_df.info()
print("\n", orders_jun_df.head(), "\n")


Guests DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   guest_id         988 non-null    int64 
 1   first_name       988 non-null    object
 2   last_name        988 non-null    object
 3   gender           988 non-null    object
 4   date_of_birth    988 non-null    object
 5   phone            988 non-null    object
 6   email            988 non-null    object
 7   referral_source  988 non-null    object
dtypes: int64(1), object(7)
memory usage: 61.9+ KB

    guest_id first_name last_name  gender date_of_birth         phone  \
0      1001        Fey    Border  Female    07/04/1994  108-471-1810   
1      1002     Nealon    Yaneev    Male    04/15/1984  985-192-5059   
2      1003     Arleta   Stanway  Female    08/07/1974  960-148-8404   
3      1004     Barbie  Gockelen  Female    05/01/1981  426-340-0038   
4      100

In [4]:
# ---------------------------
# Step 4: Reference Checks
# ---------------------------

# Combine orders into one DataFrame
orders_df = pd.concat([orders_apr_df, orders_may_df, orders_jun_df], ignore_index=True)

# 4.1 Check menu_item_id
missing_menu_ids = orders_df[~orders_df['menu_item_id'].isin(menu_df['menu_item_id'])]
print("Missing menu IDs in orders (not found in menu):")
print(missing_menu_ids)

# 4.2 Check client_id
missing_guest_ids = orders_df[~orders_df['client_id'].isin(guests_df['guest_id'])]
print("\nMissing guest IDs in orders (not found in guests):")
print(missing_guest_ids)


Missing menu IDs in orders (not found in menu):
Empty DataFrame
Columns: [order_id, client_id, order_date, order_time, menu_item_id, quantity, discount (%), total_price ($)]
Index: []

Missing guest IDs in orders (not found in guests):
Empty DataFrame
Columns: [order_id, client_id, order_date, order_time, menu_item_id, quantity, discount (%), total_price ($)]
Index: []


In [7]:
# ---------------------------
# Step 5: Date & Time Conversion
# ---------------------------

import datetime

# Convert 'order_date' from string to datetime
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format='%Y-%m-%d')

# If 'order_time' is a separate column
orders_df['order_time'] = pd.to_datetime(orders_df['order_time'], format='%H:%M:%S').dt.time


def get_time_of_day(t):
    """
    Categorize time of day into four intervals ( 08:00–23:00 only):
      - Morning:   08:00 to 11:00
      - Lunch:     11:00 to 14:00
      - Afternoon: 14:00 to 18:00
      - Evening:   18:00 to 23:00
    """

    if t < datetime.time(11, 0):
        return "Morning"     # covers 08:00–10:59
    elif t < datetime.time(14, 0):
        return "Lunch"       # covers 11:00–13:59
    elif t < datetime.time(18, 0):
        return "Afternoon"   # covers 14:00–17:59
    else:
        return "Evening"     # covers 18:00–22:59

# Convert 'order_time' to datetime.time (if not already done)
orders_df['order_time'] = pd.to_datetime(orders_df['order_time'], format='%H:%M:%S').dt.time

# Apply the new function
orders_df['time_of_day'] = orders_df['order_time'].apply(get_time_of_day)

print(orders_df[['order_time', 'time_of_day']].head(10))


  order_time time_of_day
0   08:00:00     Morning
1   08:00:00     Morning
2   08:00:00     Morning
3   08:00:00     Morning
4   08:02:00     Morning
5   08:02:00     Morning
6   08:03:00     Morning
7   08:03:00     Morning
8   08:03:00     Morning
9   08:04:00     Morning


In [8]:
# ---------------------------
# Step 6: Handling Missing Values & Duplicates
# ---------------------------

# Check for null values in combined orders
null_summary = orders_df.isnull().sum()
print("Null values per column:\n", null_summary)


# Check for duplicates
duplicates = orders_df.duplicated()
print(f"Number of duplicated rows: {duplicates.sum()}")
if duplicates.sum() > 0:
    # Remove duplicates if they are truly duplicates
    orders_df.drop_duplicates(inplace=True)
    print("Duplicates removed.")


Null values per column:
 order_id           0
client_id          0
order_date         0
order_time         0
menu_item_id       0
quantity           0
discount (%)       0
total_price ($)    0
time_of_day        0
dtype: int64
Number of duplicated rows: 0


In [9]:
# ---------------------------
# Step 7: Save Cleaned Orders
# ---------------------------

# Saveing cleaned orders for future steps
orders_df.to_csv("orders_cleaned.csv", index=False)

print("Cleaned orders saved locally (orders_cleaned.csv).")


Cleaned orders saved locally (orders_cleaned.csv).
